# Amazon SageMaker Multi-Model Endpoints using XGBoost
With Amazon SageMaker Multi-Model Endpoints (new feature under NDA), customers can create an endpoint that hosts multiple models. These Endpoints are well suited to cases where there are a large number of models that can be served from a shared inference container and when the prediction request tolerates occasional cold start latency penalties for invoking infrequently used models.

At a high level, Amazon SageMaker manages the lifetime of the models in-memory for multi-model endpoints. When an invocation request is made for a particular model, Amazon SageMaker routes the request to a particular instance, downloads the model from S3 to that instance, and loads the required model to the memory of the container. Then Amazon SageMaker performs an invocation on the model. If the model is already loaded in memory, the invocation will be fast since the downloading and loading steps are skipped.

To demonstrate how multi-model endpoints are created and used, this notebook provides an example using a set of XGBoost models that each predict housing prices for a single location. The multi-model endpoint capability is designed to work across all machine learning frameworks and algorithms including those where you bring your own container.

## Import libraries 

In [ ]:
import numpy as np
import pandas as pd
import json
import datetime
import time
from time import gmtime, strftime
import matplotlib.pyplot as plt
import os

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3

## TEMPORARY FOR BETA: Get access to the new feature in boto3

In [ ]:
!aws configure add-model --service-model file://sagemaker-2017-07-24.normal.json --service-name sagemaker-multimodel-endpoints
!aws configure add-model --service-model file://sagemaker-runtime.normal.json --service-name sagemaker-runtime-multimodel-endpoints

## Setup Variables

In [ ]:
sm_client = boto3.client(service_name='sagemaker-multimodel-endpoints')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime-multimodel-endpoints')

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
REGION     = boto3.Session().region_name
BUCKET     = sagemaker_session.default_bucket()

from sagemaker.amazon.amazon_estimator import get_image_uri
XGB_CONTAINER = get_image_uri(REGION, 'xgboost', '0.90-1')
# TEMP during beta: hard-code uri for beta container with multi-model endpoint support
#XGB_CONTAINER = '878107166805.dkr.ecr.us-east-1.amazonaws.com/preprod-xgboost-framework:mms_beta_1'
XGB_CONTAINER = '878107166805.dkr.ecr.us-west-2.amazonaws.com/preprod-xgboost-framework:mms_beta_1'

DATA_PREFIX            = 'DEMO_MME_REC'
RECOMMENDATIONS_MODEL_NAME     = 'recommendations' ##TODO : CHANGE THIS 
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

#TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

MOVIE_RECOMMENDATION_MODEL='movie-rec-all-features.tar.gz'
MUSIC_RECOMMENDATION_MODEL='music-rec-kiran.tar.gz'

MOVIE_RECOMMENDATION_TEST_DATA='movie_test.csv'
MUSIC_RECOMMENDATION_TEST_DATA='music_test.csv'

## Import models into hosting
A big difference for multi-model endpoints is that when creating the Model entity, the container's `ModelDataUrl` is the S3 prefix where the model artifacts that are invokable by the endpoint are located. The rest of the S3 path will be specified when actually invoking the model. Remember to close the location with a trailing slash.

The `Mode` of container is specified as `MultiModel` to signify that the container will host multiple models.

### Deploy model artifacts to be found by the endpoint
As described above, the multi-model endpoint is configured to find its model artifacts in a specific location in S3. For each trained model, we make a copy of its model artifacts into that location.

In our example, we are storing all the models within a single folder. The implementation of multi-model endpoints is flexible enough to permit an arbitrary folder structure. For a set of housing models for example, you could have a top level folder for each region, and the model artifacts would be copied to those regional folders.

Note that we are purposely *not* copying the first model. This will be copied later in the notebook to demonstrate how to dynamically add new models to an already running endpoint.

In [ ]:
##Get the recommendation models and test data from S3 bucket
pretrained_models_bucket_prefix="https://reinvent-sagemaker-deployment-2019.s3-us-west-2.amazonaws.com/recommendations"


!wget $pretrained_models_bucket_prefix/models/movie-recommendation-models/$MOVIE_RECOMMENDATION_MODEL -P models
!wget $pretrained_models_bucket_prefix/models/music-recommendation-models/$MUSIC_RECOMMENDATION_MODEL -P models

!wget $pretrained_models_bucket_prefix/data/movie-recommendations-data/$MOVIE_RECOMMENDATION_TEST_DATA -P data
!wget $pretrained_models_bucket_prefix/data/music-recommendations-data/$MUSIC_RECOMMENDATION_TEST_DATA -P data





In [ ]:
##Copy model to S3 bucket.
def copy_model_to_s3(model_name):
    key = os.path.join(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name)
    with open('models/'+model_name, 'rb') as file_obj:
        print("Uploading ", file_obj , " to bucket ", BUCKET, " as " , key)
        s3.Bucket(BUCKET).Object(key).upload_fileobj(file_obj)

In [ ]:
##Copy movie recommendation model to S3
copy_model_to_s3(MOVIE_RECOMMENDATION_MODEL)

### Create the Amazon SageMaker model metadata
Here we use `boto3` to establish the model metadata. Instead of describing a single model, this metadata will indicate the use of multi-model semantics and will identify the source location of all specific model artifacts.

In [ ]:
def create_multi_model_metadata(multi_model_name, role):
    # establish the place in S3 from which the endpoint will pull individual models
    _model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)
    _container = {
        'Image':        XGB_CONTAINER,
        'ModelDataUrl': _model_url,
        'Mode':         'MultiModel'
    }
    create_model_response = sm_client.create_model(
        ModelName = multi_model_name,
        ExecutionRoleArn = role,
        Containers = [_container])
    
    return _model_url

In [ ]:
name = '{}-{}'.format(RECOMMENDATIONS_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
model_url = create_multi_model_metadata(name, role)
print("model_url ", model_url)

### Create the multi-model endpoint
There is nothing special about the SageMaker endpoint config metadata for a multi-model endpoint. You need to consider the appropriate instance type and number of instances for the projected prediction workload. The number and size of the individual models will drive memory requirements.

Once the endpoint config is in place, the endpoint creation is straightforward.

In [ ]:
endpoint_config_name = name
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': ENDPOINT_INSTANCE_TYPE,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': name,
        'VariantName': 'AllTraffic'}])

endpoint_name = name
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

In [ ]:
print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print('    {}...'.format(status))
print('DONE')

## Exercise the multi-model endpoint

### Establish a predictor

Since we are using the boto3 interface above to create the endpoint config and endpoint, we use `RealTimePredictor` to get access to the endpoint for predictions.

In [ ]:
from sagemaker import RealTimePredictor

xgb_predictor = RealTimePredictor(endpoint_name)

xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [ ]:
print("Model url - ", model_url)
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls $model_url

In [ ]:
##Get recommendations from test data
def get_recommendations(model_name, test_data_file_name):
    
    print("Getting recommendations for test data in ", test_data_file_name, " using the model ", model_name)
    
    with open('data/'+test_data_file_name, 'r') as f:
        contents = f.readlines()
        
    for i in range(0, 20):

        line = contents[i]
        #print(line)
        split_data = line.split(',')
        original_value = split_data.pop(0)

        payload = ','.join(split_data)

        response = runtime_sm_client.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            TargetModel=model_name,
                            Body=payload)

        prediction = response['Body'].read().decode('utf-8')
    
        print("Original Value ", original_value , "Prediction : ", prediction)    
    

In [ ]:
##Get movie recommendations
get_recommendations(MOVIE_RECOMMENDATION_MODEL,MOVIE_RECOMMENDATION_TEST_DATA)

In [ ]:
##TODO : Need to show recommended movie names to the user.

### Dynamically deploy another model
Here we demonstrate the power of dynamic loading of new models. We purposely did not copy the first model when deploying models earlier. Now we deploy an additional model and can immediately invoke it through the multi-model endpoint. As with the earlier models, the first invocation to the new model takes longer, as the endpoint takes time to download the model and load it into memory.

In [ ]:
##Copy music recommendation model to S3
copy_model_to_s3(MUSIC_RECOMMENDATION_MODEL)

In [ ]:
print("Model url - ", model_url)
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls $model_url

In [ ]:
##Get music recommendations
get_recommendations(MUSIC_RECOMMENDATION_MODEL,MUSIC_RECOMMENDATION_TEST_DATA)

In [ ]:
##TODO : Need to show recommended song names to the user.

## Clean up
Here, to be sure we are not billed for endpoints we are no longer using, we clean up.

In [ ]:
# shut down the endpoint
xgb_predictor.delete_endpoint()

In [ ]:
# maybe delete model too
xgb_predictor.delete_model()